In [1]:
import numpy as np
import scipy.stats
import scipy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib
from tqdm import tqdm

Using matplotlib backend: TkAgg



# Testing the likelihood expression in (Lau et. al, 2017)


Suppose there are two patients in a population and $E_1$ and $E_2$ refer to the time of exposure of both patients to an infection. One of the components of the likelihood function of a given infection tree to be computed is the probability $P_E(E_1 = e_1,E_2 = e_2)$. In the paper, it is assumed that $E_1$ and $E_2$ are samples from a Poisson distribution with some time-varying rate $\lambda$. It is also assumed that once a patient is exposed, the patient is infected. Subsequent infections of the same patient don't matter. In such a scenario,  the Poisson assumption allows the above probability to be calculated rather easily as $E_1$ and $E_2 - E_1$, which are random variables that represent the waiting times for patients 1 and to be infected, are exponentially distributed random variables. 

If a Poisson infection process with rate $\lambda$ infects two nodes at times $T_i$ and $T_{i+1}$, the distribution of the difference $w = T_{i+1} - T_i$ is $P_W(W = w) = \lambda \exp( -\lambda w)$. The mean value $<W> = 1/\lambda$. 

There are two ways in which 1 and 2 can get infected. 
1. Either both independently get infected from some background source, or 
2. Patient 1 gets infected by the background source at time $E_1$. After the disease incubates for a time period $I_1$, patient 2 gets infected from patient 1 at time $E_2$ (in such a case, $E_2 > E_1 + I_1$). 



In [22]:
lambdaBase = 2 #Arrival rate of infections from background.
lambdaNode = 3 #Arrival rate of infections from patient 1
numInfections = 20000 #Number of infection time pairs (E_1,E_2) to sample in a given replicate
kdeReplicates = 20 #Number of replicates to run

## Case 1 : Background node infects both patients

In [23]:
meanDiff = []
stdDiff = []
E2evalList = np.linspace( 0.6, 2, 20 ) 
testPair = [0.5,1.8]
plt.figure()
for E2eval in E2evalList:
    testPair[1] = E2eval
    temp = []
    theoPdf = lambdaBase * lambdaBase * np.exp( -lambdaBase * testPair[1] )    
    for _ in range(kdeReplicates):
        E1fromIndex = np.random.exponential( scale=1.0/lambdaBase, size=numReplicates )
        E2fromIndex = E1fromIndex + np.random.exponential( scale=1.0/lambdaBase, size=numReplicates )
        
        E1 = E1fromIndex
        E2 = E2fromIndex
        kde = scipy.stats.gaussian_kde( (E1,E2) )
        temp.append( 100*abs(kde.evaluate( testPair )[0] - theoPdf)/theoPdf )

    meanDiff.append( np.mean( temp ) )
    stdDiff.append( np.std(temp) ) 
 
plt.errorbar( E2evalList, meanDiff, yerr=stdDiff, marker='o' )
ax = plt.gca()
ax.set_xlabel( 'Infection time of second patient' )
ax.set_ylabel( '%age difference between KDE and theoretical estimate')



## Case 2 : Background infects node 1, and node 1 infects node 2

In [24]:
meanDiff = []
stdDiff = []
E2evalList = np.linspace( 2, 3, 20 )
testPair = [0.5,1.8]
plt.figure()
I1 = 1
for E2eval in tqdm(E2evalList):
    testPair[1] = E2eval
    temp = []
    theoPdf = lambdaNode * np.exp( -lambdaNode * (testPair[1] - testPair[0] - I1) ) * lambdaBase * np.exp( -lambdaBase * testPair[0] )                
    for _ in range(kdeReplicates):
        E1fromIndex = np.random.exponential( scale=1.0/lambdaBase, size=numReplicates )
        #I1= np.random.gamma( 3, 0.02, size=numReplicates )
        E2fromPatient1 = I1 + E1fromIndex + np.random.exponential( scale=1.0/lambdaNode, size=numReplicates )

        E1 = E1fromIndex
        E2 = E2fromPatient1
        kde = scipy.stats.gaussian_kde( (E1,E2) )
        temp.append( 100*abs(kde.evaluate( testPair )[0] - theoPdf)/theoPdf )

    meanDiff.append( np.mean( temp ) )
    stdDiff.append( np.std(temp) ) 
 
plt.errorbar( E2evalList, meanDiff, yerr=stdDiff, marker='o' )
ax.set_xlabel( 'Infection time of second patient' )
ax.set_ylabel( '%age difference between KDE and theoretical estimate')

100%|██████████| 20/20 [00:00<00:00, 22.15it/s]
